In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
# Helper Module: Stochastic Depth (DropPath)
# Randomly drops a residual connection during training.
# This is a form of regularization that improves performance for deep networks.
def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks)."""
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output

In [4]:
class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks)."""
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)

# Core ViT Components
class PatchEmbedding(nn.Module):
    """
    Splits the image into patches and embeds them.
    Parameters:
        img_size (int): Size of the input image (e.g., 32 for CIFAR-10).
        patch_size (int): Size of each patch (e.g., 4 for CIFAR-10).
        in_channels (int): Number of input channels (e.g., 3 for RGB).
        embed_dim (int): The embedding dimension.
    """
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=512):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2

        # A convolution layer to convert image to patches and embed them
        self.proj = nn.Conv2d(
            in_channels,
            embed_dim,
            kernel_size=patch_size,
            stride=patch_size,
        )

    def forward(self, x):
        # Input x: (B, C, H, W)
        x = self.proj(x)  # (B, E, H/P, W/P)
        x = x.flatten(2)  # (B, E, N) where N is number of patches
        x = x.transpose(1, 2)  # (B, N, E)
        return x

class MultiHeadAttention(nn.Module):
    """Multi-Head Self-Attention block."""
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class MLP(nn.Module):
    """A simple Feed-Forward Network."""
    def __init__(self, in_features, hidden_features, out_features, drop=0.):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class TransformerEncoderBlock(nn.Module):
    """A single block of the Transformer Encoder."""
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0., attn_drop=0., drop_path=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = MultiHeadAttention(dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, out_features=dim, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class VisionTransformer(nn.Module):
    """The main Vision Transformer model."""
    def __init__(self, img_size=32, patch_size=4, in_chans=3, num_classes=10, embed_dim=512, depth=6,
                 num_heads=8, mlp_ratio=4., qkv_bias=True, drop_rate=0.1, attn_drop_rate=0.,
                 drop_path_rate=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim

        self.patch_embed = PatchEmbedding(
            img_size=img_size, patch_size=patch_size, in_channels=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.n_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks = nn.Sequential(*[
            TransformerEncoderBlock(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i])
            for i in range(depth)])
        self.norm = nn.LayerNorm(embed_dim)

        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes)

        # Initialize weights
        nn.init.trunc_normal_(self.pos_embed, std=.02)
        nn.init.trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        x = self.blocks(x)
        x = self.norm(x)

        # Use the CLS token for classification
        return self.head(x[:, 0])

In [5]:
# --- Data Loading and Training Setup ---

def get_loaders(batch_size=256):
    """Prepare CIFAR-10 DataLoaders."""
    # Data augmentation and normalization for training
    # Just normalization for validation
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)
    trainloader = DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
    testloader = DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    return trainloader, testloader

def train_one_epoch(model, dataloader, criterion, optimizer, scheduler, device, epoch):
    """Train the model for one epoch."""
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for i, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)

        # Linear warm-up
        if epoch < 5: # Warm-up for first 5 epochs
            lr_scale = (i + 1 + len(dataloader) * epoch) / (len(dataloader) * 5)
            for pg in optimizer.param_groups:
                pg['lr'] = CFG.lr * lr_scale

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar.set_postfix({
            'loss': f'{total_loss/(i+1):.4f}',
            'acc': f'{100.*correct/total:.2f}%',
            'lr': f'{optimizer.param_groups[0]["lr"]:.5f}'
        })

    if epoch >= 5: # Start scheduler after warm-up
        scheduler.step()

def evaluate(model, dataloader, criterion, device):
    """Evaluate the model on the test set."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    accuracy = 100. * correct / total
    avg_loss = total_loss / len(dataloader)
    print(f'Test Results: Accuracy: {accuracy:.2f}% | Avg Loss: {avg_loss:.4f}')
    return accuracy

In [6]:
# --- Main Execution Block ---

if __name__ == '__main__':
    # Configuration
    class CFG:
        img_size = 32
        patch_size = 4
        in_chans = 3
        num_classes = 10
        embed_dim = 512
        depth = 6
        num_heads = 8
        mlp_ratio = 2.0 # Reduced mlp_ratio for smaller model
        qkv_bias = True
        drop_rate = 0.1
        attn_drop_rate = 0.1
        drop_path_rate = 0.1

        batch_size = 256
        epochs = 100
        lr = 1e-3
        weight_decay = 0.05

    # Setup
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Data
    trainloader, testloader = get_loaders(CFG.batch_size)

    # Model
    model = VisionTransformer(
        img_size=CFG.img_size,
        patch_size=CFG.patch_size,
        in_chans=CFG.in_chans,
        num_classes=CFG.num_classes,
        embed_dim=CFG.embed_dim,
        depth=CFG.depth,
        num_heads=CFG.num_heads,
        mlp_ratio=CFG.mlp_ratio,
        qkv_bias=CFG.qkv_bias,
        drop_rate=CFG.drop_rate,
        attn_drop_rate=CFG.attn_drop_rate,
        drop_path_rate=CFG.drop_path_rate
    ).to(device)

    print(f"Model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters.")

    # Loss, Optimizer, Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    # Cosine Annealing scheduler, starts after the warm-up period
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.epochs - 5)

    # Training Loop
    best_acc = 0.0
    for epoch in range(CFG.epochs):
        train_one_epoch(model, trainloader, criterion, optimizer, scheduler, device, epoch)
        current_acc = evaluate(model, testloader, criterion, device)
        if current_acc > best_acc:
            best_acc = current_acc
            print(f"🎉 New best accuracy: {best_acc:.2f}%")
            # You can save your best model here if you want
            # torch.save(model.state_dict(), 'best_vit_cifar10.pth')

    print(f"\n--- Training Finished ---")
    print(f"🏆 Best Test Accuracy: {best_acc:.2f}%")

Using device: cuda


100%|██████████| 170M/170M [00:03<00:00, 43.1MB/s]


Model has 12,681,738 trainable parameters.


Epoch 1: 100%|██████████| 196/196 [01:31<00:00,  2.15it/s, loss=2.0955, acc=21.95%, lr=0.00020]


Test Results: Accuracy: 28.39% | Avg Loss: 1.9285
🎉 New best accuracy: 28.39%


...

Test Results: Accuracy: 84.95% | Avg Loss: 0.4420


Epoch 99: 100%|██████████| 196/196 [01:35<00:00,  2.05it/s, loss=0.5350, acc=81.05%, lr=0.00000]


Test Results: Accuracy: 84.88% | Avg Loss: 0.4429


Epoch 100: 100%|██████████| 196/196 [01:35<00:00,  2.06it/s, loss=0.5354, acc=80.98%, lr=0.00000]


Test Results: Accuracy: 84.90% | Avg Loss: 0.4429

--- Training Finished ---
🏆 Best Test Accuracy: 84.95%
